In [1]:
using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra
using InvertedIndices
using MachineLearningPotential

# Symmetry Functions

Time to write and test the basics of SymmFunctions for ML potentials

In [2]:
struct RadialType2{T} <: RadialSymmFunction{T}
    eta::T
    r_cut::T
end

In [3]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
test_symm_func = RadialType2{Float64}(0.001,11.338)

RadialType2{Float64}(0.001, 11.338)

In [4]:
function calc_one_symm_function(r2_ij,eta,r_cut)
    if r2_ij != 0 && r2_ij < r_cut^2 
        g_ij = exp(-eta*(r2_ij)) * cutoff_function( sqrt(r2_ij)/r_cut )

        return g_ij
    else
        return 0.
    end
end
function calc_symm_functions(dist2_matrix,index,symmfunc::RadialType2)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    
    gvec = calc_one_symm_function.(dist2_matrix[index,:],eta,r_cut)
    return sum(gvec)   
end
function calc_symm_functions(dist2_matrix,index,symmfunc::RadialType2,g_min,g_max)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    gvec = calc_one_symm_function.(dist2_matrix[index,:],eta,r_cut)
    g_unscaled = sum(gvec)

    return (g_unscaled - g_min)/(g_max - g_min)   
end

calc_symm_functions (generic function with 2 methods)

In [5]:
[calc_symm_functions(dis2mat,i,test_symm_func) for i in 1:55]

55-element Vector{Float64}:
 14.640666117305633
 11.966607067855739
 11.966607067541357
 11.966607067163583
 11.966607068360855
 11.966607068035858
 11.96660706756069
 11.966607067316641
 11.966607067815215
 11.96660706776792
  ⋮
  5.6995369746290425
  7.455371452799304
  5.699536974721713
  7.455371453127722
  7.455371452781338
  7.455371453231261
  7.455371452614264
  7.455371452215095
  5.699536975557825

In [6]:
[calc_symm_functions(dis2mat,i,test_symm_func,0.450564942,15.523345849) for i in 1:55]

55-element Vector{Float64}:
 0.9414388269065572
 0.7640290266879376
 0.76402902666708
 0.7640290266420167
 0.7640290267214493
 0.7640290266998875
 0.7640290266683626
 0.7640290266521712
 0.764029026685249
 0.7640290266821113
 ⋮
 0.34824177867478656
 0.4647321920234493
 0.34824177868093475
 0.46473219204523813
 0.46473219202225735
 0.46473219205210736
 0.46473219201117283
 0.46473219198469007
 0.3482417787364064

Require benchmarking concerning the use of Not(i) vs if ||

In [7]:
@benchmark calc_symm_functions($dis2mat,1,test_symm_func)

BenchmarkTools.Trial: 10000 samples with 192 evaluations.
 Range (min … max):  514.240 ns …  15.983 μs  ┊ GC (min … max): 0.00% … 95.08%
 Time  (median):     631.432 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   669.130 ns ± 687.927 ns  ┊ GC (mean ± σ):  4.75% ±  4.46%

   ▇▅█▄▄                           ▁▁▁                           
  ▅█████▇▅▃▂▂▁▁▁▂▂▂▂▂▂▁▁▁▁▁▁▂▁▃▅▇██████▆▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁ ▃
  514 ns           Histogram: frequency by time          874 ns <

 Memory estimate: 1008 bytes, allocs estimate: 3.

In [8]:
function test_calc_one_symm_func(r2_ij,eta,r_cut)
    if r2_ij < r_cut^2
        g_ij = exp(-eta*(r2_ij)) * cutoff_function( sqrt(r2_ij)/r_cut )
    else
        g_ij = 0
    end
    return g_ij
end
function test_calc_one_symm_func(r2_ij,symmfunc)
    if r2_ij < symmfunc.r_cut^2
        g_ij = exp(-symmfunc.eta*(r2_ij)) * cutoff_function( sqrt(r2_ij)/r_cut )
    else
        g_ij = 0
    end
    return g_ij
end
function test_calc_symm_functions(dist2_mat,index,symmfunc)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    g_tot = 0
    for j in (1:55)[Not(index)]
        g_tot += calc_one_symm_function(dist2_mat[index,j],eta,r_cut)
    end
    return g_tot
end

test_calc_symm_functions (generic function with 1 method)

In [9]:
@benchmark test_calc_symm_functions($dis2mat,1,test_symm_func)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.418 μs … 546.848 μs  ┊ GC (min … max): 0.00% … 98.74%
 Time  (median):     4.813 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.852 μs ±  13.960 μs  ┊ GC (mean ± σ):  6.27% ±  2.61%

   ▄█▆▄                                                        
  ▃████▆▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▃▅▅▆▆▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  4.42 μs         Histogram: frequency by time        8.78 μs <

 Memory estimate: 8.94 KiB, allocs estimate: 273.

# Angular Symmetry Function

In [10]:
function angular_measure(a,b,c,r2ij,r2ik)
    θ = (a - b)⋅(a - c)/sqrt(r2ij*r2ik) 
    return θ
end
function angular_measure(a,b,c)
    r2_ab,r2_ac,r2_bc = distance2(a,b),distance2(a,c),distance2(b,c)

    θ = angular_measure(a,b,c,r2_ab,r2_ac)
    
    return θ,r2_ab,r2_ac,r2_bc
end


angular_measure (generic function with 2 methods)

In [11]:
θ,r_ab,r_bc,r_bc = angular_measure(positions[1:3]...)

(0.44721359552456313, 20.939011641470017, 20.939011642417448, 23.14960191763878)

In [82]:
function calc_one_symm_function1(θ,r2_ij,r2_ik,r2_jk,r_cut,η,λ,ζ)
    d_cut=r_cut^2
    if r2_ij == 0. || r2_ik == 0 || r2_jk == 0 || r2_ij<d_cut || r_ik<d_cut || r_jk < d_cut 
        g = 0
    else
        g = (1+λ*θ)^ζ*exp(-η*(r2_ij+r2_ik+r2_jk))*cutoff_function(sqrt(r2_ij))*cutoff_function(sqrt(r2_ik))*cutoff_function(sqrt(r2_jk))
    end
    return g
end
#second version designed for use with the Not function to prevent zeros
function calc_one_symm_function2(θ,r2_ij,r2_ik,r2_jk,r_cut,η,λ,ζ)
    d_cut=r_cut^2
    if r2_ij > d_cut || r2_ik > d_cut || r2_jk > d_cut 
        g = 0.
    else
        g = (1+λ*θ)^ζ*exp(-η*(r2_ij+r2_ik+r2_jk))*cutoff_function(sqrt(r2_ij)/r_cut)*cutoff_function(sqrt(r2_ik)/r_cut)*cutoff_function(sqrt(r2_jk)/r_cut)
    end
    return g
end


calc_one_symm_function2 (generic function with 1 method)

In [81]:
function calc_one_symm_func(position1,position2,position3,symmfunc)
    θ,r2_ij,r2_ik,r2_jk = angular_measure(position1,position2,position3)
    g = calc_one_symm_function2(θ,r2_ij,r2_ik,r2_jk,symmfunc.r_cut,symmfunc.eta,symmfunc.lambda,symmfunc.zeta)
    return g
end

function calc_one_symm_func(position1,position2,position3,r2_ij,r2_ik,r2_jk,symmfunc)
    θ = angular_measure(position1,position2,position3,r2_ij,r2_ik)
    g = calc_one_symm_function2(θ,r2_ij,r2_ik,r2_jk,symmfunc.r_cut,symmfunc.eta,symmfunc.lambda,symmfunc.zeta)
    return g
end



calc_one_symm_func (generic function with 2 methods)

Different methods for the creation of our inputs theta r_ijk

In [14]:
[angular_measure(positions[1],positions[j],positions[k]) for j =(1:55)[Not(1)], k in (1:55)[Not(1)]]

54×54 Matrix{NTuple{4, Float64}}:
 (1.0, 20.939, 20.939, 0.0)             …  (-1.0, 20.939, 82.1728, 186.072)
 (0.447214, 20.939, 20.939, 23.1496)       (-0.447214, 20.939, 82.1728, 140.213)
 (0.447214, 20.939, 20.939, 23.1496)       (-0.447214, 20.939, 82.1728, 140.213)
 (0.447214, 20.939, 20.939, 23.1496)       (-0.447214, 20.939, 82.1728, 140.213)
 (0.447214, 20.939, 20.939, 23.1496)       (-0.447214, 20.939, 82.1728, 140.213)
 (0.447214, 20.939, 20.939, 23.1496)    …  (-0.447214, 20.939, 82.1728, 140.213)
 (-0.447214, 20.939, 20.939, 60.6064)      (0.447214, 20.939, 82.1728, 66.0106)
 (-0.447214, 20.939, 20.939, 60.6064)      (0.447214, 20.939, 82.1728, 66.0106)
 (-0.447214, 20.939, 20.939, 60.6064)      (0.447214, 20.939, 82.1728, 66.0106)
 (-0.447214, 20.939, 20.939, 60.6064)      (0.447214, 20.939, 82.1728, 66.0106)
 ⋮                                      ⋱  
 (-0.447214, 82.1728, 20.939, 140.213)  …  (0.447214, 82.1728, 82.1728, 90.848)
 (-0.525731, 62.2806, 20.939, 121.19)    

In [15]:
X = [angular_measure(positions[1],positions[j],positions[k]) for j =(1:55)[Not(1)] for k in (1:j-1)[Not(1)]]

1431-element Vector{NTuple{4, Float64}}:
 (0.44721359552456313, 20.939011642417448, 20.939011641470017, 23.14960191763878)
 (0.44721359556787066, 20.939011647492432, 20.939011641470017, 23.149601918630534)
 (0.447213595632537, 20.939011647492432, 20.939011642417448, 23.149601916446162)
 (0.44721359554352996, 20.93901163737478, 20.939011641470017, 23.149601914056962)
 (-0.4472135953692125, 20.93901163737478, 20.939011642417448, 60.60644463770368)
 (0.44721359555272444, 20.93901163737478, 20.939011647492432, 23.149601917001032)
 (0.4472135954940274, 20.939011637125667, 20.939011641470017, 23.14960191599233)
 (-0.4472135955566442, 20.939011637125667, 20.939011642417448, 60.60644464519245)
 (-0.4472135954422881, 20.939011637125667, 20.939011647492432, 60.60644464774802)
 (0.4472135955054501, 20.939011637125667, 20.93901163737478, 23.149601913250173)
 ⋮
 (0.525731112086319, 82.17275004373793, 62.28061741180172, 69.2332443521747)
 (0.4472135954067908, 82.17275004373793, 82.1727500494146, 90.

In [83]:
thetavec = [angular_measure(positions[1],positions[j],positions[k],dis2mat[1,j],dis2mat[1,k]) for j =(1:55)[Not(1)] for k in (1:55)[Not(1,j)]]

2862-element Vector{Float64}:
  0.44721359552456313
  0.44721359556787066
  0.44721359554352996
  0.4472135954940274
  0.4472135955111943
 -0.4472135954185562
 -0.4472135955111942
 -0.44721359556301754
 -0.4472135955317425
 -0.44721359538853644
  ⋮
  0.525731112086319
  0.4472135954067908
  0.5257311120705334
  0.44721359549679773
  0.8506508083755363
  0.850650808344319
  0.8506508083545801
  0.8506508083579555
  0.8506508083348291

In [159]:
function calc_test_symmetry_function(positions,dis2_mat,index,symmfunc)

    g_vec = [calc_one_symm_func(positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],symmfunc) for j=(1:55) if j !=index for k= 1:j-1 if k != index]
    g = 2^(1-symmfunc.zeta)*sum(g_vec)
    return g_vec
end


calc_test_symmetry_function (generic function with 1 method)

In [160]:
test_symm_funtion = AngularType3{Float64}(0.0001,11.338,1.,1.)

AngularType3{Float64}(0.0001, 11.338, 1.0, 1.0)

In [161]:
g = calc_test_symmetry_function(positions,dis2mat,1,test_symm_funtion)

1431-element Vector{Float64}:
 0.37424307343045743
 0.37424307339074536
 0.3742430734181402
 0.3742430735080577
 0.05149494652903834
 0.37424307343497204
 0.37424307348086117
 0.0514949464990291
 0.051494946499458474
 0.3742430735414329
 ⋮
 0.004986007195418177
 0.0007965771436781896
 0.004986007195316935
 0.0007965771441450275
 0.022982721794237975
 0.02298272178951796
 0.022982721797201816
 0.022982721793819567
 0.0229827217856787

In [162]:
sum(g)

35.92620108172107

In [163]:
testindex = rand(1:55)

42

In [164]:
g2 = calc_test_symmetry_function(positions,dis2mat,2,test_symm_funtion)

1431-element Vector{Float64}:
 0.3945473580289051
 0.3945473579483398
 0.3510792449328997
 0.3945473581246414
 0.05825958460258176
 0.35107924494522086
 0.3945473581166109
 0.058259584576470284
 0.05825958456791025
 0.3510792450085504
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [165]:
sum(g2)

26.73978507341531